# <center> Parallel Fast Fourier Method for Computing Matrix Vector Products </center>

<center>By Jack Hamel</center>

<img src="https://upload.wikimedia.org/wikipedia/commons/b/b8/Fftw-logo-med.gif" width="20%">
<p style="text-align: right;">Image from: https://en.wikipedia.org/wiki/FFTW</p>

---
# Abstract

My project's original intent was to parallelize Fast Fourier Transforms in a quantum-electromagnetics code called QuEST.  This was derailed when it was dicovered that the code is not in a funtional state.  QuEST is not my own and therefore could have been a potentially monumental task to fix myself so it was deemed necessary to redirect the goal of this project.  In order to preserve the work done during the software exploration, I decided to take the key mechanism that I wanted to parallelize in QuEST and write my own standalone version to parallelize.  This mechanism is the foundation to fast-fourier based methods in computational science.  The idea is that a matrix-vector product can be effected using a fourier transform based operation, if the matrix involved is circulant.  I wrote a code to compute a matrix vector product in a such a fashion and parallelized it using FFTW3 and OpenMP.  The results of this project are that the fully parallelized code performs fastest for matrices larger than $10^4x10^4$.



---
# Methodology

The first part of this project was the software exploration where I wrote example programs using FFTW3 and OpenMP.  The example programs were to serve as references during part 2 of the project. The OpenMP example uses a parallel for and OpenMP functions to set max threads, check total threads used, and check thread IDs. The concepts here are not new to me as we learned them in class, but are now consolidated into a simple code for reference.  The FFTW example fills an array with a sinusoid and computes the 1D Discrete Fourier Transform (DFT) of it serially and in parallel.  For more details on these example programs and how to compile them, see my software exploration report.

Before explaining the deliverables on this project, I will present an overview of the mathematics involved.  Given an $NxN$ matrix $A_c$, which is circulant, and a size $N$ vector $x$, we can take a matrix vector product of the two like so: 
$$ A_cx=b $$
We could compute each element of $b$ with a naive approach by multiplying each matrix row times $x$ and summing those products but because $A_c$ is circulant, we are allowed to use a faster approach.  We can instead do the following:
$$\mathcal{F}^{-1}\{diag(\mathcal{F}\{a_c\})\mathcal{F}\{x\}\} = b$$
where $\mathcal{F}$ is the Fourier matrix and $a_c$ is one column of the circulant matrix, $A_c$.  Because Fourier Transforms can be computated using Fast Fourier Transforms which scale as $O(NlogN)$, we can compute $b$ exactly, in much less time.  In my codes, I am interested in only the time it takes to compute a matrix vector product using these two methods.  I do not include any operations related to setting up these calculations in my measured run times (e.g. filling vectors).

The codes I wrote to study the effects of parallelizing an FFT based matrix vector product began with writing a non-FFT based program called **matvec_slow.cpp**.  This program does the traditional matrix-vector product mentioned just previously.  This was mainly done to provide a solution I can compare my FFT codes to while they are being developed to ensure there are no bugs.  It was also useful in providing data to compare the FFT codes to.  The second code I wrote was a serial FFT based code called **matvec_serial.cpp**.  This code computed $b$ using the FFT method to effect a matrix-vector product with one thread.  The third code, **matvec_threads.cpp** did the same, but distributed the FFT operations across multiple threads.  The last code I wrote was the same as the third code, but now had additional parallelization using OpenMP and is called **matvec_parallel.cpp**.  Parallel for loops were introduced around a loop in the set up code which did not affect my measured run times, but did drastically reduce the time required to set up my arrays.  The second parallelized loop was the loop to compute $diag(\mathcal{F}\{a_c\})\mathcal{F}\{x\}$.  This was intended to reduce the measured run time.

These codes are all part of my submission and can be built by calling the make command with their makefile in the same directory as the source code.  You may also compile each code on its own by calling make with the word following "\_" and preceding ".cpp" of its corresponding source code file name (e.g. **make serial** to compile **matvec_serial.cpp**).  Lastly, **make clean** will clean your directory of all executables associated with the matvec codes.





---
# Results

I conducted two different timing studies.  The first study show the performance of all four codes for various matrix sizes.  The parallel codes were using 16 threads.

<img src="./size_time_plot.png" width="80%">

As expected, for matrices $N<200$ we see the traditional unparallelized approach performing the best (note that obtaining data for high matrix sizes using this approach requires ridiculous amounts of memory and would not be insightful, even if obtained).  This makes sense as the overhead associated with computing FFTs is quite noticable at those scales.  Once we get larger, we see the serial FFT based code performing the best.  This is likely due to the FFT overhead becoming less significant compared to the transform calculations.  For $N>10^4$ we start to see the fully parallelized (FFTW and OpenMP) with 16 threads takes over as the fastest algorithm and holds this position indefinitely.  The Parallel code with only parallel FFTs does not beat the serial code until $N>10^6$ and continues to scale the same as the full parallel code.  The full parallel code is faster than the parallel FFT code for all matrix sizes used in this study because of the added speedup due to OpenMP.  A strange behavior in this study is the lack of scaling of the Parallel codes until they hit a large enough matrix.  This behavior is likely due to computation time being dominated by FFTW overhead; rather than the actual calculations.


The second study shows the performance of the two parallel codes for different numbers of threads ranging from 1 to 32.  The matrix size used for this study was $10^7x10^7$.

<img src="./threads_time_plot.png" width="80%">

The results of this study show that the full parallel code scales much better with the number of threads.  This is easily explained by the fact that there are more parallelized components of the code and therefore more to benefit from multiple threads.  The scaling is not linear because this is not a 100% efficient parallelization.  There are components that are not parallelized and there is overhead that parallelization adds that decreases performance.  The only interesting phenomenon here is the leveling out between 16 and 32 threads.  This was investigated and can be explained by the matrix size.  If $N=10^8$, for example, there is no significant leveling out between 16 and 32 threads.  If the matrices are large enough, we continue to see benefit in increasing the number of threads.  My hypothesis is that added overhead and inefficiency with increased thread numbers is not as significant for larger and larger matrix sizes.

---
# Future Work and Concluding Discussion

To summarize, this project was about parallelizing a fast method for computing a matrix vector product using Fast Fourier Transforms.  I wrote four programs: a traditional matvec code, a serial FFT code, a parallel FFT code, and a parallel FFT code with OpenMP as well.  I found that for very small matrices ($N<200$) the traditional approach is fastest.  For matrices with $200<N<10^4$, the serial FFT algorithm is the fastest.  With any matrix size larger, the full parallel code with parallel FFTs and parallel loops is the fastest.  There is no feasible scenario to use the parallel code without parallel loops.  Furthermore, maximizing the number of threads used may not always be the best choice.  The speed increases associated with more threads levels out for a fixed matrix size and no more benefit is gained.  

The mathematics and OpenMP were already familiar to me prior to doing this project so I learned the most about using FFTW3.  FFTW3 was completely new to me prior.  I focused on using the library to compute 1-dimensional DFTs both serially and then in parallel.  The idea of using FFTW3 is as follows:

1) Set up FFTW3 objects to be transformed

2) Create FFT plan which dictates what transform is computed and how

3) Execute plan

4) Destroy plan

The only addition to this process when running with multiple threads is that before step 2, you must tell the library to plan with $n$ threads, $n$ being any number you choose.  Admittedly, parallelization is the easiest part of using FFTW3 since it requires only one more line of code.  

I added two separate OpenMP loops in my code.  One was used during set up to parallelize the filling of my FFTW3 objects (1-dimensional complex arrays in this case).  I ran into issues with serialization here because I was attempting to fill the array with randomly generated numbers and usage of my random number generator was serializing this operation.  This was solved by giving each thread its own private generator.

Additionally, although not much of a challenge, it was fun combining multiple parallelization libraries to maximize the performance of my code.  Adding OpenMP to my parallel FFT code gave dramatic speed increases that would be silly not to use to solve this problem.  

Future experiments with this program would be to program it to run on GPUs.  I think computing the FFTs on a GPU as well as doing multiplication on GPUs would yield the fastest version of this code.  

---
# References

Serial FFTs: http://www.fftw.org/fftw3_doc/FFTW-Reference.html#FFTW-Reference

Parallel FFTs: http://www.fftw.org/fftw3_doc/Multi_002dthreaded-FFTW.html